# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Te Anau,-45.4167,167.7167,69.10,54,100,5.91,NZ,1612754873
1,1,Cape Town,-33.9258,18.4232,66.99,72,0,14.97,ZA,1612754841
2,2,Narsaq,60.9167,-46.0500,39.20,28,20,39.12,GL,1612754878
3,3,Hobart,-42.8794,147.3294,66.20,45,40,18.41,AU,1612754747
4,4,Petropavlovsk-Kamchatskiy,53.0452,158.6483,24.80,80,90,8.95,RU,1612754863


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [9]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

humidity = cities_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
# Criteria: Temp between 65 - 75, humidity < 50%, winds < 17 mph, cloudiness <= 40%

nice_cities = cities_df.loc[(cities_df['Max Temp'] <= 75) & (cities_df['Max Temp'] >= 65) 
               & (cities_df['Humidity'] < 50) & (cities_df['Wind Speed'] < 17) & (cities_df['Cloudiness'] <= 40)]
nice_cities

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,Mangan,27.5167,88.5333,68.05,21,0,3.83,IN,1612754905
141,141,Nago,26.5881,127.9761,71.60,46,1,12.66,JP,1612755120
235,235,Māngrol,21.1167,70.1167,73.81,28,0,9.55,IN,1612755287
239,239,Burnie,-41.0667,145.9167,71.01,40,17,3.74,AU,1612755298
274,274,Along,28.1667,94.7667,69.26,26,0,0.63,IN,1612755358
320,320,Nishihara,26.1842,127.7558,71.60,37,20,14.97,JP,1612755438
393,393,Makkah al Mukarramah,21.4267,39.8261,68.32,25,0,15.26,SA,1612755571
428,428,Atoyac,20.0167,-103.5333,68.00,28,1,8.05,MX,1612755632
448,448,Cooma,-36.2333,149.1333,71.60,49,17,11.50,AU,1612755671
490,490,Pālanpur,24.1667,72.4333,74.23,20,0,5.99,IN,1612755750


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = nice_cities
hotel_df["Hotel Name"] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add keyword to params dict
    params['location'] = str(hotel_df.loc[index,"Lat"]) + "," + str(hotel_df.loc[index,"Lng"])

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    hotel_df.loc[index,"Hotel Name"] = results[0]['name']

C:\Users\Kelly\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kelly\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locationss = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locationss.values, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))